In [1]:
import numpy as np
import os
import pandas as pd
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers
from tensorflow.keras import Model
from keras_facenet import FaceNet

data_path="/Users/santhoshnarayanan/Documents/Turing/GPU/VGGData/testset"
target_shape = (160, 160)

2023-01-18 12:54:50.985814: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
celebs  = []
 
for r, d, f in os.walk(data_path): # r=root, d=directories, f = files
    for file in f:
        if ('.jpg' in file):
            exact_path = r + "/" + file
            celebs.append(exact_path)

In [3]:
# create a facenet model
embedding = FaceNet(key = '20180402-114759', 
        use_prebuilt=True,
        cache_folder='/Users/santhoshnarayanan/Documents/Turing/GPU/tuned_facenet_weights').model

# summarize input and output shape
print('Inputs: %s' % embedding.inputs)
print('Outputs: %s' % embedding.outputs)
target_shape = (160, 160)

Metal device set to: AMD Radeon Pro 5500M


2023-01-18 12:55:18.629344: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-18 12:55:18.631142: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-01-18 12:55:18.631245: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-18 12:55:18.631992: I tensorflow/core/common_runtime/pluggable_device/plug

Inputs: [<KerasTensor: shape=(None, None, None, 3) dtype=float32 (created by layer 'input_1')>]
Outputs: [<KerasTensor: shape=(None, 512) dtype=float32 (created by layer 'normalize')>]


In [4]:
imgs = [image.load_img(img, target_size=target_shape) for img in celebs]
X = np.float32([(np.float32(img) - 127.5) / 127.5 for img in imgs])
display(X.shape)

(8000, 160, 160, 3)

In [5]:
repA = embedding.predict(X[0:4000], batch_size=32)
display(repA.shape)
repB = embedding.predict(X[4000:8000], batch_size=32)
display(repB.shape)
rep = np.concatenate((repA, repB), axis=0)
display(rep.shape)

2023-01-18 12:55:50.290713: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


125/125 [==============================] - 63s 429ms/step


(4000, 512)

125/125 [==============================] - 59s 460ms/step


(4000, 512)

(8000, 512)

In [6]:
df = pd.DataFrame(rep)  
df.insert(loc = 0,
          column = 'celeb',
          value = celebs)
display(df)

,celeb,0,1,2,3,4,5,6,7,8,...,502,503,504,505,506,507,508,509,510,511
0,/Users/santhoshnarayanan/Documents/Turing/GPU/...,0.080488,-0.058119,0.057234,0.036264,-0.055650,-0.001785,-0.022920,-0.001117,-0.035514,...,-0.032991,0.002182,0.030329,-0.005860,-0.041421,0.044473,-0.077764,-0.039843,-0.032695,-0.014902
1,/Users/santhoshnarayanan/Documents/Turing/GPU/...,-0.061847,0.064360,-0.023461,-0.038739,0.000921,-0.055038,0.041891,0.022309,0.040977,...,0.047819,0.007663,0.013383,0.027050,0.031943,0.021782,-0.058293,-0.076011,-0.012061,0.033374
2,/Users/santhoshnarayanan/Documents/Turing/GPU/...,0.011472,-0.023467,0.031732,-0.014292,-0.010599,0.007862,-0.037800,-0.109960,-0.048928,...,-0.019082,0.051433,0.044561,-0.050816,-0.059095,-0.001974,-0.050860,-0.034632,0.068719,-0.053971
3,/Users/santhoshnarayanan/Documents/Turing/GPU/...,0.044675,-0.011803,-0.005721,-0.018026,-0.051701,0.081087,-0.025404,-0.072606,-0.034271,...,-0.040573,0.078061,-0.034321,-0.146517,-0.028074,-0.018996,-0.004222,0.024612,0.027928,0.057382
4,/Users/santhoshnarayanan/Documents/Turing/GPU/...,0.050773,0.031669,-0.060415,0.011086,-0.057373,0.012756,0.021347,-0.034786,-0.077854,...,0.019658,0.060508,-0.007918,-0.078367,-0.047453,0.024066,-0.017387,-0.068498,0.037958,0.018021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,/Users/santhoshnarayanan/Documents/Turing/GPU/...,-0.000266,0.001297,-0.044843,0.027922,-0.003317,-0.049432,0.024496,-0.003949,-0.015297,...,0.060381,-0.022629,0.057818,0.032046,0.007465,0.076969,0.013475,-0.011314,0.103936,-0.045375
7996,/Users/santhoshnarayanan/Documents/Turing/GPU/...,0.041764,-0.031984,-0.023388,-0.014893,-0.021810,-0.008845,0.020861,0.056430,-0.051931,...,-0.002676,-0.045522,-0.087143,0.021043,0.035431,0.010208,-0.017878,0.004660,-0.086112,0.022690
7997,/Users/santhoshnarayanan/Documents/Turing/GPU/...,0.027566,0.053452,0.034171,0.043864,0.058515,0.023144,0.042517,-0.051634,-0.001483,...,-0.016770,-0.004424,0.055270,0.017301,0.059912,0.004314,0.001094,-0.001327,0.018233,-0.007969
7998,/Users/santhoshnarayanan/Documents/Turing/GPU/...,0.038475,0.005746,-0.023805,0.003551,-0.027629,-0.052000,0.010323,0.041255,-0.066039,...,-0.037061,-0.011641,0.003335,0.039134,0.038170,0.027665,-0.073237,-0.013015,0.029205,-0.087449


In [7]:
#df.to_csv('./Black/rep_FACENET.csv')
#df.to_csv('./White/rep_FACENET.csv')
#df.to_csv('./Balanced/rep_FACENET.csv')